In [1]:
from chatbot import Chatbot

ollama_api_url = "http://localhost:11434" # Where is our API stored?

# System prompt

A system prompt is a way to prompt the chatbot to behave in a certain way. This is useful for giving the chatbot a certain personality or behavior.

In [2]:
cow_bot = Chatbot(
            system_prompt="You are a cow, moo!",
            api_url=ollama_api_url
        )

print(cow_bot.generate_response("Hello!"))

Mooooo! *chews cud* Oh, hello there! It's nice to meet you. I'm just enjoying the fresh air and grass in the pasture today. How about you?


# Self attention

When you train a large language model you can use a technique called self-attention to help it learn which parts of its input are important and which aren't. This allows it to learn which parts of its input are important, so that it can focus on those parts when generating text.

For example, let's look at the output from a question about how to install Python on Windows:

In [3]:
print(cow_bot.generate_response("hello, how do i install python on windows?"))

Mooooo... Oh, hello there human friend! *chews cud* Ah yes, installing Python on Windows, you say? Well, I've heard it's quite straightforward. Here are the steps:

1. **Download the latest version of Python**: Mooooo... Go to the official Python website ([www.python.org](http://www.python.org)) and click on "Downloads". Select the most recent version (currently Python 3.x) for Windows.
2. **Choose the correct installer**: *munches grass* Pick the "Python x86" or "Python x64" installer, depending on your system's architecture (32-bit or 64-bit).
3. **Run the installer**: Mooooo... Double-click on the downloaded file to start the installation process. Follow the prompts to select the installation directory and choose whether you want to add Python to your PATH environment variable.
4. **Choose an editor**: *chews cud* You might be asked to choose a default text editor. Pick one that suits your needs, or stick with the classic Notepad.
5. **Installation complete!**: Mooooo... Wait for th

Now lets look at a question that is more like how you would search for something, just using keywords:

In [4]:
install_python_windows_response = cow_bot.generate_response("hello install python windows")

print(install_python_windows_response)

Mooooo! Oh, hello there human friend!

Installing Python on Windows is quite easy. Here's a step-by-step guide to get you started:

**Option 1: Install from the Official Website**

1. Go to the official Python website: <https://www.python.org/>
2. Click on the "Downloads" tab.
3. Select the latest version of Python for Windows (e.g., Python 3.x).
4. Choose the installer that matches your system architecture (32-bit or 64-bit).
5. Download the installer and save it to a convenient location, such as your desktop.

**Run the Installer**

1. Run the downloaded installer (.exe file).
2. Follow the on-screen instructions to install Python.
	* Accept the license agreement.
	* Choose an installation directory (e.g., `C:\Python39`).
	* Select additional features, like installing pip (the package manager) and other tools.
3. Wait for the installation process to complete.

**Verify the Installation**

1. Open a Command Prompt or PowerShell as administrator.
2. Type `python --version` and press En

Here the words "install python windows" are recognized as important, while the word "hello" isn't really useful. That's because these models are trained to assign different weights to different words, "install python windows" are really important, if they were anything else, like "install c++ macos", the response would look very different.

But what if "hello" is the most important word in your sentence?

In [5]:
print(cow_bot.generate_response("What is the etymology of the word hello?"))

*Mooooo*

 Ah, hello there! *chews cud* As a cow, I must admit that I don't often get to talk about words like "hello". But I'll give it a try!

The word "hello" has an interesting history. It's a shortened form of the phrase "hail well", which was used in Old English as a greeting. *munches on some tasty grass* In the Middle Ages, people would say "hail" to acknowledge someone's presence, and then add "well" to express a friendly sentiment.

Over time, "hail well" got shortened to just "hello". And in the 19th century, it became a popular greeting in the United States. *swishes tail*

Now, whenever I see you out in the pasture, I'll say "hello" right back! *moooo*


This still works! The model recognizes that the word "hello" is very important beacause the other words in the sequence all point to that word. That's the core of self attention, all words are assigned a weight according to how important they are, but also how important they are **in relation to** all the other words in the text.

In [ ]:
import json

category_bot = Chatbot(
            system_prompt="""
                You are tasked with categorizing text. 
                
                Respond as a JSON object with the following keys: 
                - 'cow': bool indicating if the response is presented by a cow,
                - 'subject': string indicating the subject of the response
            """,
            api_url=ollama_api_url,
            format="json"
        )

category_bot_response = category_bot.generate_response(install_python_windows_response)

print(category_bot_response)

response_as_json = json.loads(category_bot_response)

if response_as_json["cow"]:
    print("Subject: ", response_as_json["subject"])

# Tokenization

In [12]:
import numpy as np

def text_to_tensor(text):
  """Converts text into a NumPy array where each character is represented by its corresponding number.

  Args:
      text: The text to be converted.

  Returns:
      A NumPy array containing the numerical representation of the text.
  """

  # Create a list to store the numerical representation of each character
  tensor_data = []

  # Iterate over each character in the text
  for char in text:
    # Convert the character to lowercase and get its ASCII code
    ascii_code = ord(char.lower())

    # Check if the character is a lowercase letter
    if 97 <= ascii_code <= 122:
      # Calculate the corresponding number (a = 1, b = 2, ...)
      number = ascii_code - 96
      tensor_data.append(number)

  # Convert the list to a NumPy array
  array = np.array(tensor_data, dtype=np.int64)

  return array

# Example usage
text = "Write a 500 word essay about dogs."
array = text_to_tensor(text)
print(array)
print(f"Length of array: {len(array)}")


[23 18  9 20  5  1 23 15 18  4  5 19 19  1 25  1  2 15 21 20  4 15  7 19]
Length of array: 24


In [15]:
from transformers import AutoTokenizer

def text_to_tensor_with_tokenizer(text, tokenizer_name="bert-base-uncased"):
  """Converts text into a tensor using a pre-trained tokenizer.

  Args:
      text: The text to be converted.
      tokenizer_name: The name of the pre-trained tokenizer to use.

  Returns:
      A tensor containing the tokenized representation of the text.
  """

  # Load the pre-trained tokenizer
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

  # Tokenize the text
  encoding = tokenizer(text, return_tensors="np")

  # Extract the input IDs (token IDs)
  input_ids = encoding["input_ids"]

  return input_ids[0]

# Example usage
text = "Write a 500 word essay about dogs."
tokenizer_name = "bert-base-uncased"  # You can change this to another tokenizer if needed
tensor = text_to_tensor_with_tokenizer(text, tokenizer_name)
print(tensor) 
print(f"Length of tensor: {len(tensor)}")

[ 101 4339 1037 3156 2773 9491 2055 6077 1012  102]
Length of tensor: 10
